**IMPORT LIBRARY**

In [31]:
# import thư viện
import requests
import time
import random
import pandas as pd
import concurrent.futures
import os
import csv
import json
from tqdm import tqdm
from IPython.display import clear_output

In [32]:
#Khai báo header
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 6.3; Win64; x64; rv:83.0) Gecko/20100101 Firefox/83.0',
    'Accept': 'application/json, text/plain, */*',
    'Accept-Language': 'vi-VN,vi;q=0.8,en-US;q=0.5,en;q=0.3',
    'Referer': 'https://tiki.vn/?src=header_tiki',
    'x-guest-token': '8jWSuIDBb2NGVzr6hsUZXpkP1FRin7lY',
    'Connection': 'keep-alive',
    'TE': 'Trailers',
}

In [33]:
params = {
    'limit': '48',
    'include': 'sale-attrs,badges,product_links,brand,category,stock_item,advertisement',
    'aggregations': '1',
    'trackity_id': '70e316b0-96f2-dbe1-a2ed-43ff60419991',
    'category': '1883',
    'page': '1',
}

In [34]:
def read_src_urlkey_from_txt(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()
    pairs = [line.strip().split(',') for line in lines]
    return pairs

In [35]:
def fetch_product_ids(page, src, urlKey):
    params['page'] = page
    params['src'] = src
    params['urlKey'] = urlKey
    response = requests.get('https://tiki.vn/api/v2/products', headers=headers, params=params)
    product_ids = []
    
    if response.status_code == 200 and response.json().get('data'): 
        print(f'Request success on page {page} for {src}, {urlKey}!!!')
        
        for record in response.json().get('data'):
            product_ids.append({'id': record.get('id')})
    else:
        if response.status_code != 200:
            print(f'Request failed on page {page} for {src}, {urlKey} with status code {response.status_code}.')
        elif not response.json().get('data'):
            print(f'No more data on page {page} for {src}, {urlKey}.')

    return product_ids, response.status_code, bool(response.json().get('data'))

In [36]:
def parser_product(json):
    d = dict()
    d['id'] = json.get('id')
    d['master_id'] = json.get('master_id')
    d['sku'] = json.get('sku')
    d['all_time_quantity_sold'] = json.get('all_time_quantity_sold')

    breadcrumbs = json.get('breadcrumbs')
    if breadcrumbs:
        first_breadcrumb = breadcrumbs[0]  # Get the first item in the breadcrumbs list
        d['breadcrumb_name'] = first_breadcrumb.get('name')  # Get the 'name' of the first breadcrumb
        d['breadcrumb_category_id'] = first_breadcrumb.get('category_id') 

    quantity_sold = json.get('quantity_sold')
    if quantity_sold is not None:
        d['quantity_sold'] = quantity_sold.get('value')

    d['rating_average'] = json.get('rating_average')
    d['review_count'] = json.get('review_count')
    d['original_price'] = json.get('original_price')
    d['price'] = json.get('price')
    d['discount'] = json.get('discount')
    d['discount_rate'] = json.get('discount_rate')

    d['list_price'] = json.get('list_price')
    d['price_usd'] = json.get('price_usd')
    d['discount'] = json.get('discount')
    d['discount_rate'] = json.get('discount_rate')
    
    quantity_sold = json.get('quantity_sold')
    if quantity_sold is not None:
        d['quantity_sold'] = quantity_sold.get('value')
        
    d['review_count'] = json.get('review_count')
    d['order_count'] = json.get('order_count')
    d['inventory_status'] = json.get('inventory_status')
    d['is_visible'] = json.get('is_visible')
    d['product_name'] = json.get('name')

    brand = json.get('brand')
    if brand is not None:
        d['brand_id'] = brand.get('id')
        d['brand_name'] = brand.get('name')
    
    categories_name = json.get('categories')
    if categories_name is not None:
        d['categories_name'] = categories_name.get('name')
     
    d['description_short'] = json.get('short_description')
    d['product_links'] = json.get('product_links')
    d['day_ago_created'] = json.get('day_ago_created')
        
    inventory = json.get('inventory')
    if inventory is not None:
        d['fulfillment_type'] = inventory.get('fulfillment_type')
        d['product_virtual_type'] = inventory.get('product_virtual_type')

    d['inventory_status'] = json.get('inventory_status')    
    return d

In [37]:
def download_and_parse(pid):
    try:
        response = requests.get('https://tiki.vn/api/v2/products/{}'.format(pid), headers=headers, params=params)
        if response.status_code == 200 and response.text.strip():
            return parser_product(response.json())
        else:
            print(f"Received status code {response.status_code} for product ID {pid} or response body is empty")
            return None
    except (TypeError, ValueError) as e:  # Catch errors related to parsing JSON
        print(f"Error parsing JSON for product ID {pid}: {str(e)}")
        return None

In [38]:
pairs = read_src_urlkey_from_txt('./Data/Categories.txt') 


In [39]:
for src, urlKey in pairs:
    product_id = []
    empty_data_count = 0  # Counter for empty data
    page = 1
    while True:
        if empty_data_count >= 10:
            break
        with concurrent.futures.ThreadPoolExecutor(max_workers=100) as executor:
            future = executor.submit(fetch_product_ids, page, src, urlKey)
            result, status_code, data_exists = future.result()
            product_id.extend(result)
            if status_code != 200 or not data_exists:
                empty_data_count += 1
            else:
                empty_data_count = 0  # Reset the counter if there's data
            page += 1

    df = pd.DataFrame(product_id)
    # Save the file with utf-8-sig encoding
    df.to_csv(f'./Data/{urlKey}_product_id.csv', index=False, encoding='utf-8-sig')

    print('Completed crawling successfully for src={}, urlKey={}'.format(src, urlKey))
    time.sleep(5)  # Sleep for 5 seconds between crawling different sources

Request success on page 1 for c873,  sach-kien-thuc-tong-hop!!!
Request success on page 2 for c873,  sach-kien-thuc-tong-hop!!!
Request success on page 3 for c873,  sach-kien-thuc-tong-hop!!!
Request success on page 4 for c873,  sach-kien-thuc-tong-hop!!!
Request success on page 5 for c873,  sach-kien-thuc-tong-hop!!!
Request success on page 6 for c873,  sach-kien-thuc-tong-hop!!!
Request success on page 7 for c873,  sach-kien-thuc-tong-hop!!!
Request success on page 8 for c873,  sach-kien-thuc-tong-hop!!!
Request success on page 9 for c873,  sach-kien-thuc-tong-hop!!!
Request success on page 10 for c873,  sach-kien-thuc-tong-hop!!!
Request success on page 11 for c873,  sach-kien-thuc-tong-hop!!!
Request success on page 12 for c873,  sach-kien-thuc-tong-hop!!!
Request success on page 13 for c873,  sach-kien-thuc-tong-hop!!!
Request success on page 14 for c873,  sach-kien-thuc-tong-hop!!!
Request success on page 15 for c873,  sach-kien-thuc-tong-hop!!!
Request success on page 16 for c87

In [40]:
p_ids = df['id'].tolist()

In [43]:
# Use ThreadPoolExecutor
result = []
lines_printed = 0
clear_after = 50  # Clear output after this many lines
with concurrent.futures.ThreadPoolExecutor(max_workers=5) as executor:
    futures = {executor.submit(download_and_parse, pid): pid for pid in p_ids}
    
    # Init tqdm bar
    with tqdm(total=len(p_ids)) as pbar:
        for i, future in enumerate(concurrent.futures.as_completed(futures)):
            pid = futures[future]
            try:
                data = future.result()
                if data is not None:
                    result.append(data)
                    print('Crawled {} items successfully'.format(i + 1))
                    lines_printed += 1
                # Clear output after every 'clear_after' lines
                if lines_printed >= clear_after:
                    clear_output(wait=True)
                    lines_printed = 0
            except Exception as exc:
                print('%r generated an exception: %s' % (pid, exc))
            # Update tqdm bar
            pbar.update()
            
df_product = pd.DataFrame(result)
df_product.to_csv('./Data/product_data.csv', index=False, encoding='utf-8-sig')

 94%|█████████▍| 1888/2000 [02:39<00:15,  7.29it/s]

Crawled 1889 items successfully

 94%|█████████▍| 1890/2000 [02:39<00:14,  7.77it/s]


Crawled 1890 items successfully


 95%|█████████▍| 1892/2000 [02:39<00:14,  7.54it/s]

...

Error parsing JSON for product ID 192473147: Expecting value: line 1 column 1 (char 0)
Crawled 1993 items successfully
Error parsing JSON for product ID 112218352: Expecting value: line 1 column 1 (char 0)
Error parsing JSON for product ID 150849958: Expecting value: line 1 column 1 (char 0)
Error parsing JSON for product ID 32376222: Expecting value: line 1 column 1 (char 0)


100%|█████████▉| 1999/2000 [02:45<00:00, 17.03it/s]

Crawled 1997 items successfully
Error parsing JSON for product ID 88069618: Expecting value: line 1 column 1 (char 0)
Error parsing JSON for product ID 52785428: Expecting value: line 1 column 1 (char 0)


100%|██████████| 2000/2000 [02:45<00:00, 12.10it/s]

Error parsing JSON for product ID 165085520: Expecting value: line 1 column 1 (char 0)


In [42]:
df_product.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1027 entries, 0 to 1026
Data columns (total 27 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   id                      1027 non-null   int64  
 1   master_id               1027 non-null   int64  
 2   sku                     1027 non-null   object 
 3   all_time_quantity_sold  984 non-null    float64
 4   breadcrumb_name         1027 non-null   object 
 5   breadcrumb_category_id  1027 non-null   int64  
 6   quantity_sold           984 non-null    float64
 7   rating_average          1027 non-null   float64
 8   review_count            1027 non-null   int64  
 9   original_price          1027 non-null   int64  
 10  price                   1027 non-null   int64  
 11  discount                1027 non-null   int64  
 12  discount_rate           1027 non-null   int64  
 13  list_price              1027 non-null   int64  
 14  price_usd               0 non-null      